In [66]:
import requests
import sqlite3 as sql
import bs4 as bs
import re
import datetime 

base_url = 'https://www.imdb.com'

In [67]:
# Create a database to store queries
# Use Variables to store all table names for portability.
table_name = 'TVSeries'
column_names = {table_name:['USER_EMAIL','TVSERIES']}
database_name='user_tv_series.db'

conn = sql.connect(database_name)
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS {}'.format(table_name)+'({0} TEXT NOT NULL,{1} TEXT)'.format(*column_names[table_name]))

In [68]:
# take input
email = 'soham'# input("Enter email address:")
tv_string = 'Game of thrones, suits, friends, black mirror, gotham' # input("Enter TV series names seperated by a string")

In [69]:
tv_string = tv_string.replace(' ','+')
tv_list = tv_string.split(',')

In [70]:
def search_tv_series_soup(tv_series_title):
    search_link = base_url + '/search/title'
    params = {'title':tv_series_title,'title_type':'tv_series'}
    tvr = requests.request('GET', url = search_link, params = params)
    tvr_soup = bs.BeautifulSoup(tvr.text,'lxml')
    return tvr_soup

In [71]:
# This function returns link to the page of given tv series.
def find_tv_series_page_link(tv_series_title):
    if type(tv_series_title)!= str:
        print('Invalid argument data type passed to find_link function')
    
    # get search results in html
    tvr_soup = search_tv_series_soup(tv_series_title)
    tv_temp = tv_series_title.lower().replace('+','')
    tv_temp = tv_series_title.lower().replace(' ','')
    for a in tvr_soup.findAll('a', href= True):
        stemp = str(a.string)
        if tv_temp in stemp.lower().replace(' ',''):
            link = a['href']
            break
    return base_url+link[0:link.find('?')-1]

In [72]:
find_tv_series_page_link('friends   ')
 

'https://www.imdb.com/title/tt0108778'

In [73]:
def get_episodes_page(tv_series_title, s=-1):
    link = find_tv_series_page_link(tv_series_title)
    params = {'season': s}
    eps_resp = requests.request('GET', link + '/episodes', params = params)
#     print(eps_resp.url)
    eps_soup = bs.BeautifulSoup(eps_resp.text, 'lxml')
    return eps_soup
a = get_episodes_page('friends')

In [74]:
# This function gets episode date in datetime format from the html value passed to it
def get_ep_date(ep):
    ep_text = ep.text.replace(' ','')
    list_ep_date = ep_text.split()[0]
    try:
        ep_date = datetime.datetime.strptime(ep_text,'\n%d%b.%Y\n')
    except ValueError:
        try:
            ep_date = datetime.datetime.strptime(ep_text,'\n%d%b%Y\n')
        except ValueError:
            ep_date = datetime.datetime.strptime(ep_text,'\n%Y\n')
    return ep_date.date()

In [75]:
def get_final_schedule(tv_series_title):
    now_date = datetime.datetime.now().date()
    episode_list_page_soup = get_episodes_page(tv_series_title)

    list_ep = episode_list_page_soup.findAll('div',{'class':'airdate'})
    i = -1
    while len(list_ep[i].string)<=1:
        i -=1
    last_ep_date = get_ep_date(list_ep[i])
    first_ep_date = get_ep_date(list_ep[0])
    if last_ep_date<now_date:
        return ("The tv series- {} has finished streaming its episodes in {} {}".format(tv_series_title, last_ep_date.strftime('%B'), last_ep_date.year))
    elif first_ep_date>now_date:
        return ("The next season of the tv series- {} begins in {}".format(tv_series_title, first_ep_date.year))
    else:
        for ep in list_ep:
            ep_date = get_ep_date(ep)
            if ep_date>=now_date:
                return("Next Episode of {} would stream on {} {} {}".format(tv_series_title, ep_date.day, ep_date.strftime('%B'), ep_date.year))  
        return ("The tv series- {} has finished streaming its episodes in {} {}".format(tv_series_title, ep_date.strftime('%B'), ep_date.year))

In [76]:
get_final_schedule('game of thrones')

'The next season of the tv series- game of thrones begins in 2019'